<a href="https://colab.research.google.com/github/worldofaryavart/colab_notebooks/blob/colabnotebook/tunehyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
 (img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
img_train = img_train.astype('float32')/ 255.0
img_test = img_test.astype('float') /255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape = (28,28)))

  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss =keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

In [ ]:
tuner.search(img_train, label_train, epochs = 50, validation_split = 0.2, callbacks = [stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 30 Complete [00h 00m 49s]
val_accuracy: 0.8705000281333923

Best val_accuracy So Far: 0.8918333053588867
Total elapsed time: 00h 12m 21s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs = 50, validation_split = 0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print("Best epoch: %d" % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5007 - accuracy: 0.8235 - val_loss: 0.4064 - val_accuracy: 0.8581
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3742 - accuracy: 0.8650 - val_loss: 0.3840 - val_accuracy: 0.8604
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3338 - accuracy: 0.8764 - val_loss: 0.3709 - val_accuracy: 0.8677
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3091 - accuracy: 0.8857 - val_loss: 0.3297 - val_accuracy: 0.8771
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2922 - accuracy: 0.8910 - val_loss: 0.3437 - val_accuracy: 0.8800
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2752 - accuracy: 0.8982 - val_loss: 0.3216 - val_accuracy: 0.8863
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2623 - accuracy: 0.9031 - val_loss: 0.3370 - val_accuracy:

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split = 0.2)

Epoch 1/40
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5040 - accuracy: 0.8236 - val_loss: 0.4013 - val_accuracy: 0.8557
Epoch 2/40
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3761 - accuracy: 0.8642 - val_loss: 0.3930 - val_accuracy: 0.8519
Epoch 3/40
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3367 - accuracy: 0.8769 - val_loss: 0.3602 - val_accuracy: 0.8689
Epoch 4/40
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3107 - accuracy: 0.8838 - val_loss: 0.3266 - val_accuracy: 0.8807
Epoch 5/40
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2897 - accuracy: 0.8926 - val_loss: 0.3277 - val_accuracy: 0.8831
Epoch 6/40
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2753 - accuracy: 0.8981 - val_loss: 0.3230 - val_accuracy: 0.8864
Epoch 7/40
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2619 - accuracy: 0.9017 - val_loss: 0.3396 - val_accuracy:

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]: ", eval_result)